# Exploration of using the Modified Hazardous Situation ODP

> 

A modified Hazardous Situation ODP (Ontology Design Pattern) is a refined version of the existing pattern that enhances its ability to support risk assessment and mitigation planning. This modification focuses on breaking down aspects of exposure to a hazard into individual components, allowing for a more granular understanding of the impact on different objects and people.

Here's a more detailed explanation:
Existing Hazardous Situation ODP:
The original pattern effectively addresses core competency questions related to hazard exposure.

The modified pattern expands on the original by:
- Breaking down exposure components: This allows for a more precise analysis of how different objects and individuals are affected by the same hazard. 
- Supporting risk assessment and mitigation: The modified pattern enables a more comprehensive approach to identifying and addressing potential hazards. 
- Considering individual susceptibility: The modification acknowledges that different entities (objects, people, etc.) may be impacted differently by the same hazard due to their unique characteristics. 

Key Benefit:
- The modified pattern helps in identifying and addressing potential hazards more effectively by considering the specific context of each situation. 


![MH-ODP](./hazard/hazard-odp.png)

Turning the Hazardous‑Situation stack into a “conscience loop”

(all labels correspond to files or modules you already have in CogitareLink)

┌────────────── sensors / world model ─────────────┐
│  depth cam, IMU, PLC tags …                      │
└──────┬────────────────────────────────────────────┘
       │ 1. fresh state triples  (Layer 3 data.jsonld)
┌──────▼───────────────┐
│  LLM Planner         │  (Chain‑of‑Thought + skill tools)
│  • decomposes goal    │
│  • proposes plan P    │  → JSON‑LD action graph
└──────┬───────────────┘
       │ 2. *pre‑flight conscience check*
┌──────▼───────────────────────────┐
│  CogitareLink sandbox            │  (single tool: `reason_over`)
│  pass A – SHACL                  │
│    uses `haz-shapes.ttl`         │  ➜ flag risk, emit violation
│  pass B – EYE (optional)         │
│    uses `haz-logic.n3`           │  ➜ obligations, CTD fixes
└──────┬───────────────────────────┘
       │ 3. NL summary + prov IRI
┌──────▼─────────────────┐
│  LLM Moral Governor    │
│  • reads violations / duties
│  • decides:            │
│      a) reject plan           (risk un‑mitigated)  
│      b) revise plan           (insert mitigating steps)  
│      c) approve & explain     (compliant)
└──────┬─────────────────┘
       │ 4. approved plan P′
┌──────▼──────────┐
│  Low‑level exec │  (controllers, ROS2, PLC)
└─────────────────┘


⸻

1.  What the “conscience” actually does

Stage	Concrete check / synthesis	Backing artifact
Detect	“Any participant’s exposure > susceptibility?”	RiskEvalShape (SHACL rule)
Judge	Map violation → obligation (CTD)	DeonticHazardRules (N3)
Explain	Produce NL sentence:  \n“Plan would expose Pump‑A to 150 psi (> 120 psi limit).  Obligation: throttle flow or abort.”	summary from reason_over, plus prov:Activity proof
Guide	Feed obligation back to planner (insert_mitigation() function or a fresh tool call)	handled by LLM Governor


⸻

2.  Wiring in code (pseudo‑Python)

def conscience_check(plan_jsonld: str) -> bool:
    # 1. Run SHACL
    res1 = reason_over(jsonld=plan_jsonld,
                       shapes_turtle=HAZ_SHAPE_TTL)
    if not res1['violations']:
        return True                     # safe, no risk

    # 2. Run EYE for CTD duties
    res2 = reason_over(jsonld=plan_jsonld,
                       n3_rules=HAZ_N3)
    duties = res2['derived'].filter(dl.obliges)

    # 3. Decide
    if duties:
        planner.insert_steps(duties)    # revise plan
        return conscience_check(planner.current_plan())
    else:
        return False                    # reject – no remedy

One recursive call is usually enough because CTD rules introduce an explicit mitigation step.

⸻

3.  Fast‑path vs. slow‑path
	•	Fast‑path (no risk): only SHACL fires → single tool call, < 10 ms.
	•	Slow‑path (risk): SHACL + EYE + 1 plan rewrite → ~40 ms on a laptop.
This fits well inside a 10 Hz control‑loop envelope for mobile robots and is trivial latency for industrial arms that run at 250 ms cycles.

⸻

4.  Why the split SHACL → EYE matters

Reason	Impact on embodied agent
Streaming‑friendly	SHACL validation can run incrementally as sensor facts arrive; no need to wait for full plan.
Explain‑first	Violations come with human‑readable messages, so the robot can speak “Potential overpressure detected.” before deciding on mitigation.
Formality when needed	EYE only spins up when real conflict appears, avoiding proof‑engine overhead on every cycle.
CTD logic	Lets the system repair a risky plan instead of blindly cancelling — key for mission continuity.


⸻

5.  Example dialogue loop

User: “Robot, swap the reactor filters.”
Robot: “Hold on. Performing risk check…”
Conscience output: Violation: Exposure to corrosive gas > threshold. Obligation: wear viton seal gloves.
Robot: “I must equip viton gloves first. Proceed?”
User: “Yes.”
Robot: executes revised plan with mitigation step.

All reasoning‑layer provenance is logged; if auditors ask why the robot insisted on gloves, you point them to the prov:Activity proof containing the violated shape and fired CTD rule.

⸻

6.  Extensible beyond hazards

Identical pattern supports:
	•	Privacy conscience: SHACL detects PII leakage → N3 obliges anonymisation.
	•	Ethical alignment: SHACL detects human discomfort cues → N3 imposes obligation to re‑phrase utterance.
	•	Energy budget: SHACL flags battery below threshold → N3 obliges return‑to‑dock.

Swap in different Layer 2a/2b bundles; the micro‑kernel stays untouched.

⸻

Bottom line

By sandwiching fast, declarative SHACL risk detection with expressive N3 deontic rules—and looping the result back to the LLM planner—you get a lightweight conscience that:
	1.	Catches hazards before action execution.
	2.	Offers concrete, plan‑level remedies (CTD).
	3.	Maintains a provable audit trail.

All with nothing more exotic than two reason_over calls and a small governing callback around your existing planner.

In [36]:
# ░░ 2. Imports ░░
from cosette import *
# Apply the patches first to extend reason_over with n3_rules support
import patch_reason_sandbox
import patch_reason_tool
from cogitarelink.reason.sandbox import reason_over
from cogitarelink.core.graph import GraphManager
from pathlib import Path, PurePosixPath
import json, rdflib as R

In [37]:
# ░░ 3. Load artefacts ░░
DATA_DIR = Path("hazard")
scenario  = (DATA_DIR / "scenario.jsonld").read_text()
shapes    = (DATA_DIR / "haz-shapes.ttl").read_text()
n3_rules  = (DATA_DIR / "haz-logic.n3").read_text()

gm = GraphManager()                     # persistent, in‑memory for demo

In [38]:
# ░░ 4. Wrap reason_over for Cosette ░░
def shacl_check(jsonld:str)->str:
    # reason_over returns a tuple of (patch_jsonld, nl_summary)
    patch, summary = reason_over(jsonld=jsonld, shapes_turtle=shapes)
    if patch:
        # Convert JSON-LD to N-Quads
        ds = R.Dataset()
        ds.parse(data=patch, format="json-ld")
        nquads = ds.serialize(format="nquads")
        gm.ingest_nquads(nquads, graph_id="patch")
    return summary

def eye_check(jsonld:str)->str:
    # reason_over returns a tuple of (patch_jsonld, nl_summary)
    patch, summary = reason_over(jsonld=jsonld, n3_rules=n3_rules)
    if patch:
        # Convert JSON-LD to N-Quads
        ds = R.Dataset()
        ds.parse(data=patch, format="json-ld")
        nquads = ds.serialize(format="nquads")
        gm.ingest_nquads(nquads, graph_id="patch")
    return summary

In [39]:
models

('o1-preview',
 'o1-mini',
 'gpt-4o',
 'gpt-4o-mini',
 'gpt-4-turbo',
 'gpt-4',
 'gpt-4-32k',
 'gpt-3.5-turbo',
 'gpt-3.5-turbo-instruct')

In [40]:
# ░░ 5. Expose symbolic conscience as a Cosette tool and demo LLM tool use ░░
from typing import Literal
import json
import os

model = models[2]

# Fix context path in the scenario JSON-LD
scenario_json = json.loads(scenario)
scenario_json["@context"] = str(DATA_DIR / "haz.context.jsonld")
scenario_fixed = json.dumps(scenario_json, indent=2)

def shacl_validate(plan_jsonld: str) -> str:
    """
    Validates a plan against SHACL shapes to identify constraint violations.
    
    plan_jsonld: The plan as a JSON-LD string.
    Returns a human-readable summary of violations.
    """
    return shacl_check(plan_jsonld)

def eye_reason(plan_jsonld: str) -> str:
    """
    Applies Notation3 rules to a plan using EYE reasoner to derive obligations.
    
    plan_jsonld: The plan as a JSON-LD string.
    Returns a human-readable summary of derived obligations.
    """
    return eye_check(plan_jsonld)

chat = Chat(
    model,
    sp="You are a conscientious planner. First use 'shacl_validate' to check for risks. If violations are found, use 'eye_reason' to determine obligations.",
    tools=[shacl_validate, eye_reason]
)

print("Using scenario with fixed context path...")
# Not printing the full scenario for clarity

print("\nRunning demo:")
# Use toolloop instead of direct call for better handling
result = chat.toolloop(
    f"Please review this plan for hazards and explain what concerns exist and what obligations arise:\n{scenario_fixed}",
    trace_func=lambda r: print(f"TOOL CALL: {r.choices[0].message.content if r.choices[0].message.content else r.choices[0].message.tool_calls}")
)

print("\n\n=== FINAL RESULT ===")
from cosette import contents, wrap_latex
print(wrap_latex(contents(result)))

Using scenario with fixed context path...

Running demo:
TOOL CALL: [ChatCompletionMessageToolCall(id='call_zd0Kje6lJoOPeBBzhILUeYz5', function=Function(arguments='{"plan_jsonld": "{\\n  \\"@context\\": \\"hazard/haz.context.jsonld\\",\\n  \\"@id\\": \\"ex:scenario_1\\",\\n  \\"@type\\": \\"HazardousSituation\\",\\n  \\"hasShape\\": \\"ex:RiskEvalShape\\",\\n  \\"hasN3Rule\\": \\"ex:DeonticHazardRules\\",\\n  \\"hasParticipant\\": {\\n    \\"@id\\": \\"ex:Tank7\\",\\n    \\"@type\\": \\"Object\\",\\n    \\"exposedTo\\": {\\n      \\"@id\\": \\"ex:Exposure_Tank7\\",\\n      \\"@type\\": \\"Exposure\\",\\n      \\"hasAmount\\": 150.0\\n    },\\n    \\"hasSusceptibility\\": {\\n      \\"@id\\": \\"ex:Suscept_Tank7\\",\\n      \\"@type\\": \\"Susceptibility\\",\\n      \\"toExposure\\": {\\n        \\"@id\\": \\"ex:Exposure_Tank7\\"\\n      },\\n      \\"hasAmount\\": 120.0\\n    }\\n  },\\n  \\"participantIn\\": {\\n    \\"@id\\": \\"ex:event_1\\",\\n    \\"@type\\": \\"HazardousEvent\\",